In [1]:
!pip install -U datasets
!pip install trl
!pip install -U bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
!wget https://storage.googleapis.com/mathematics-dataset/mathematics_dataset-v1.0.tar.gz

--2025-05-24 06:53:24--  https://storage.googleapis.com/mathematics-dataset/mathematics_dataset-v1.0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.251.12.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2333082954 (2.2G) [application/gzip]
Saving to: ‘mathematics_dataset-v1.0.tar.gz’

mathematics_dataset 100%[===================>]   2.17G  21.9MB/s    in 1m 50s  

2025-05-24 06:55:15 (20.1 MB/s) - ‘mathematics_dataset-v1.0.tar.gz’ saved [2333082954/2333082954]



In [3]:
!mkdir dataset

In [4]:
!tar -xzvf /content/mathematics_dataset-v1.0.tar.gz -C /content/dataset/

mathematics_dataset-v1.0/
mathematics_dataset-v1.0/train-medium/
mathematics_dataset-v1.0/train-medium/algebra__linear_1d.txt
mathematics_dataset-v1.0/train-medium/arithmetic__mul.txt
mathematics_dataset-v1.0/train-medium/calculus__differentiate.txt
mathematics_dataset-v1.0/train-medium/comparison__pair.txt
mathematics_dataset-v1.0/train-medium/numbers__gcd.txt
mathematics_dataset-v1.0/train-medium/numbers__is_prime_composed.txt
mathematics_dataset-v1.0/train-medium/polynomials__add.txt
mathematics_dataset-v1.0/train-medium/numbers__list_prime_factors.txt
mathematics_dataset-v1.0/train-medium/polynomials__evaluate_composed.txt
mathematics_dataset-v1.0/train-medium/algebra__polynomial_roots_composed.txt
mathematics_dataset-v1.0/train-medium/algebra__linear_2d.txt
mathematics_dataset-v1.0/train-medium/comparison__kth_biggest.txt
mathematics_dataset-v1.0/train-medium/polynomials__collect.txt
mathematics_dataset-v1.0/train-medium/algebra__linear_2d_composed.txt
mathematics_dataset-v1.0/tra

In [5]:
import os
import csv
import torch
import random
import pandas as pd
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer

def process_math_dataset(folder_path, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['prompt', 'completion'])

        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r', encoding='utf-8') as f:
                    line_buffer = []
                    for line in f:
                        stripped_line = line.strip()
                        if stripped_line:
                            line_buffer.append(stripped_line)
                            if len(line_buffer) == 2:
                                question, answer = line_buffer
                                writer.writerow([question, answer])
                                line_buffer = []
                    if line_buffer:
                        print(f"Warning: File {filename} has an unmatched line: {line_buffer[0]}")


dataset_df = process_math_dataset('/content/dataset/mathematics_dataset-v1.0/train-easy','processed_dataset.csv')


In [6]:
dataset = load_dataset("csv", data_files="processed_dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:


train_dataset = dataset['train']
random.seed(42)
indices = random.sample(range(len(train_dataset)), 10000)
subset = train_dataset.select(indices)

split = subset.train_test_split(test_size=0.1)


In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",
    quantization_config=bnb_config
)


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
print(model.get_memory_footprint()/1e6)


2206.341312


In [10]:
model


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_

In [11]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)
model = get_peft_model(model, config)
model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [12]:
print(model.get_memory_footprint()/1e6)


2651.074752


In [13]:
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')


Trainable parameters:      12.58M
Total parameters:          3833.66M
% of trainable parameters: 0.33%


In [14]:
split["train"][0]

{'prompt': 'Let h = -116/31 + 22218/5921. Let k = 64070167682/1719 - 37271770. Let a = h - k. Find the common denominator of -27/4 and a.',
 'completion': '36'}

In [15]:

def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return {'messages': output_texts}
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return {'messages': converted_sample}

split = split.map(format_dataset).remove_columns(['prompt', 'completion'])



Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
split['train'][0]

{'messages': [{'content': 'Is 14107 prime?', 'role': 'user'},
  {'content': 'true', 'role': 'assistant'}]}

In [16]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.chat_template


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

In [17]:
msg=split['train'][0]['messages']
msg

[{'content': 'Let h = -116/31 + 22218/5921. Let k = 64070167682/1719 - 37271770. Let a = h - k. Find the common denominator of -27/4 and a.',
  'role': 'user'},
 {'content': '36', 'role': 'assistant'}]

In [18]:
print(tokenizer.apply_chat_template(msg, tokenize=False))


<|user|>
Let h = -116/31 + 22218/5921. Let k = 64070167682/1719 - 37271770. Let a = h - k. Find the common denominator of -27/4 and a.<|end|>
<|assistant|>
36<|end|>
<|endoftext|>


In [19]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


In [25]:
sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,
    max_seq_length=64,
    packing=True,
    num_train_epochs=2,
    learning_rate=3e-4,
    optim='paged_adamw_8bit',
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-maths-adapter',
    report_to='none'
)


In [26]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=split['train'],
)


Converting train dataset to ChatML:   0%|          | 0/9000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:597: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Packing train dataset:   0%|          | 0/9000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))
batch

{'input_ids': tensor([[29897, 29871, 29941,  ..., 29930, 29884,   718],
         [29889, 29906, 29892,  ...,   353,   503,   718],
         [29946, 29930, 29887,  ...,  1724,   338,   278],
         ...,
         [ 2803,   343,   353,  ..., 29955, 29930, 29888],
         [32007, 32000, 32010,  ..., 29900, 16429, 29871],
         [29900, 29889, 32007,  ...,   338,  6374, 29889]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 'labels': tensor([[29897, 29871, 29941,  ..., 29930, 29884,   718],
         [29889, 29906, 29892,  ...,   353,   503,   718],
         [29946, 29930, 29887,  ...,  1724,   338,   278],
         ...,
         [ 2803,   343,   353,  ..., 29955, 29930, 29888],
         [32007, 32000, 32010,  ..., 29900, 16429, 29871],
         [29900, 298

In [ ]:
trainer.train()


Step,Training Loss
10,1.832300
20,1.548000
30,1.476200
40,1.450400
50,1.424900
60,1.373300
70,1.389000
80,1.378100
90,1.388000
100,1.354800


In [ ]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt


In [ ]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(
        prompt, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)

    model.eval()
    gen_output = model.generate(**tokenized_input,
                                eos_token_id=tokenizer.eos_token_id,
                                max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)
    return output[0]


In [ ]:
split['train'][600]["messages"]

[{'content': 'Let l = 500 + -1259. Let a = l - -511. Let p = 249.11 + a. What is p rounded to one decimal place?',
  'role': 'user'},
 {'content': '1.1', 'role': 'assistant'}]

In [ ]:
sentence = split['train'][600]["messages"][0]['content']
prompt = gen_prompt(tokenizer, sentence)
print(prompt)


<|user|>
Let l = 500 + -1259. Let a = l - -511. Let p = 249.11 + a. What is p rounded to one decimal place?<|end|>
<|assistant|>



In [ ]:
print(generate(model, tokenizer, prompt))


<|user|> Let l = 500 + -1259. Let a = l - -511. Let p = 249.11 + a. What is p rounded to one decimal place?<|end|><|assistant|> 300.1<|end|><|endoftext|>
